# **Fraud Detection Project - CA2**

## **1. Introduction**

### Project Overview
This project is the second phase (CA2) of a comprehensive fraud detection system for **SP-Buy**, an e-commerce platform. Building on the work done in **CA1**, which involved data ingestion, exploratory data analysis (EDA), and the creation of an interactive dashboard, this phase focuses on developing a machine learning model to predict fraudulent orders. The goal is to deploy this model in a user-friendly application that the operations team can use to proactively identify and mitigate fraud.

### Problem Statement
Fraudulent activities pose a significant threat to e-commerce platforms, leading to financial losses, reputational damage, and customer dissatisfaction. Detecting fraud in real-time is challenging due to the imbalanced nature of the data (fraud cases are rare compared to legitimate transactions) and the evolving tactics of fraudsters. In this project, we aim to build a robust machine learning model that can accurately identify fraudulent orders based on historical data.

### CA1 1 Recap
In **CA1 1**, we performed extensive exploratory data analysis (EDA) and data cleaning on the provided datasets:
- **Customer Features**: Information about customers, such as their order history and verification status.
- **Order Features**: Details about each order, including payment method and order value.
- **Labels**: Fraud labels indicating whether an order was fraudulent.

The cleaned and preprocessed data was used to create an interactive dashboard for monitoring fraud trends and patterns. This dashboard provided valuable insights into the dataset, enabling stakeholders to understand the nature of fraud on the platform.

### CA2 Objectives
In **CA2**, we shift our focus to **model development** and **deployment**. The key objectives are:
1. **Advanced Data Analysis**:
   - Perform additional EDA to identify feature importance and detect outliers, which are critical for model creation.
2. **Model Development**:
   - Train, evaluate, and optimize machine learning models to predict fraudulent orders.
3. **Experiment Tracking**:
   - Use **MLflow** to track experiments, log parameters, and compare model performance.
4. **Deployment**:
   - Develop a **Tkinter-based GUI application** to allow the operations team to make predictions on new orders.
5. **Automation**:
   - Design the workflow to be modular and scalable, enabling future integration with **Airflow** for automation.

### Key Challenges
- **Imbalanced Data**: Fraud cases are rare, making it challenging to train a model that can accurately detect them.
- **Feature Engineering**: Identifying and creating meaningful features that improve model performance.
- **Deciding how to proceed with experiments**: Balancing the need for thorough experimentation with time constraints was a key challenge. We addressed this by prioritizing techniques likely to have the most impact (e.g., handling imbalanced data, feature engineering).

### Structure of the Report
This report documents the entire process of **CA2**, from advanced data analysis and model development to deployment and automation. The following sections provide a detailed breakdown of each step:
- **Exploratory Data Analysis (EDA)**: Additional analysis focusing on feature importance and outlier detection.
- **Feature Engineering**: Creation of new features and their impact on model performance.
- **Data Preprocessing**: Techniques used to prepare the data for modeling.
- **Model Training and Evaluation**: Development and comparison of machine learning models.
- **Deployment**: Development of the GUI application and integration of the final model.
- **Conclusion**: Summary of findings, challenges, and future work.


---
## **2. Preparing the Dataset and Libraries**

---

#### Import Necessary Libraries

In [8]:
import pandas as pd
import numpy as np
import os

from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.compose import ColumnTransformer

import warnings
from urllib.parse import urlparse

import mlflow
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import itertools



# Show all columns
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

#### Importing the datasets

In [9]:
# %%time

# ## SQL Server Name and Database Name
# server = 'yj\SQLEXPRESS'
# database = 'PAI_CA1'

# ## Create a connection to the SQL Server
# engine = create_engine('mssql+pyodbc://{}/{}?driver=ODBC Driver 17 for SQL Server'.format(server, database))

# customer_df = pd.read_sql('SELECT * FROM [dbo].[clean-data-customer_v1.0]', engine)
# order_df = pd.read_sql('SELECT * FROM [dbo].[clean-data-order_v1.0]', engine)
# label_df = pd.read_sql('SELECT * FROM [dbo].[clean-data-label_v1.0]', engine)


# # Drop index column
# customer_df.drop(columns=['index'], inplace=True)
# order_df.drop(columns=['index'], inplace=True)
# label_df.drop(columns=['index'], inplace=True)

# # merge the data
# merged_df = pd.merge(label_df, customer_df, on=['customer_id', 'country_code'], how='inner')
# merged_df = pd.merge(merged_df, order_df, on=['order_id', 'country_code'], how='inner')

# merged_df.to_csv('merged_data.csv', index=False)

data = pd.read_csv('./data/merged_data.csv')

merged_df=data.copy()

In [10]:
merged_df.head()

,country_code,order_id,customer_id,is_fraud,mobile_verified,num_orders_last_50days,num_cancelled_orders_last_50days,num_refund_orders_last_50days,total_payment_last_50days,num_associated_customers,first_order_datetime,collect_type,payment_method,order_value,num_items_ordered,refund_value,order_date
0,BD,w2lx-myz3,bdpr8uva,0,True,0,0,0,0.000000,3,2022-08-13 03:53:52,delivery,PayOnDelivery,8.664062,9,0.870117,2023-04-08
1,BD,ta7z-r91q,bd59rlzo,0,True,7,0,0,228.042468,4,2022-05-08 14:29:19,delivery,CreditCard,21.859375,4,2.279297,2023-02-13
2,BD,t5af-wgb2,bd6zhjvq,0,True,4,1,0,45.674685,2,2021-08-25 07:47:00,delivery,AFbKash,7.125000,1,2.349609,2023-03-06
3,BD,sibu-9lm4,bd4fv4rb,0,True,19,0,3,279.805231,5,2021-12-06 13:53:22,delivery,CreditCard,4.535156,5,0.150024,2023-01-29
4,BD,we61-omtr,bdzeepq7,0,True,30,6,4,107.067610,5,2020-07-04 11:45:39,delivery,PayOnDelivery,3.011719,1,3.750000,2023-01-16


#### Convert to appropriate dtypes after importing daset

In [18]:
def convert_dtypes(df):
    # Convert 'order_value' and 'refund_value' to float16 for memory efficiency
    df['order_value'] = df['order_value'].astype('float32')
    df['refund_value'] = df['refund_value'].astype('float32')
    
    # Convert 'num_items_ordered' to uint8 after rounding
    df['num_items_ordered'] = df['num_items_ordered'].astype(float).round().astype('uint8')
    
    # Convert 'order_date' and 'first_order_datetime' to datetime
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['first_order_datetime'] = pd.to_datetime(df['first_order_datetime'])
    
    # Convert categorical columns to category dtype for efficiency
    df[['country_code', 'collect_type', 'payment_method']] = df[['country_code', 'collect_type', 'payment_method']].astype('category')
    
    # Convert numerical columns (those that represent counts or numeric features) to uint16
    df[['num_orders_last_50days', 'num_cancelled_orders_last_50days', 'num_refund_orders_last_50days']] = df[['num_orders_last_50days', 'num_cancelled_orders_last_50days', 'num_refund_orders_last_50days']].astype('uint16')
    
    # Convert 'num_associated_customers' to uint8 for efficient memory usage
    df['num_associated_customers'] = df['num_associated_customers'].astype('uint8')
    
    # Convert 'total_payment_last_50days' to float16 for memory efficiency
    df['total_payment_last_50days'] = df['total_payment_last_50days'].astype('float32')
    
    # Convert 'mobile_verified' and 'is_fraud' columns to boolean (mapping string values)
    # df['mobile_verified'] = df['mobile_verified'].map({'True': True, 'False': False})
    # df['is_fraud'] = df['is_fraud'].map({'1': True, '0': False})
    
    return df

# Mermory before
print(f'Memory usage before conversion: {merged_df.memory_usage().sum() / 1e6} MB')
merged_df = convert_dtypes(merged_df)
# Mermory after
print(f'Memory usage after conversion: {merged_df.memory_usage().sum() / 1e6} MB')

Memory usage before conversion: 140.349328 MB
Memory usage after conversion: 144.876688 MB


In [12]:
merged_df.dtypes

country_code                              category
order_id                                    object
customer_id                                 object
is_fraud                                     int64
mobile_verified                               bool
num_orders_last_50days                      uint16
num_cancelled_orders_last_50days            uint16
num_refund_orders_last_50days               uint16
total_payment_last_50days                  float16
num_associated_customers                     uint8
first_order_datetime                datetime64[ns]
collect_type                              category
payment_method                            category
order_value                                float16
num_items_ordered                            uint8
refund_value                               float16
order_date                          datetime64[ns]
dtype: object

---

## **3. Exploratory Data Analysis (EDA)**

---


2.1 Overview
In **CA1**, we performed initial exploratory data analysis (EDA) to understand the structure and distribution of the dataset. This included:

- Merging the datasets (customer-features.csv, order-features.csv, labels.csv).

- Cleaning the data (e.g., handling missing values, removing duplicates).

- Visualizing the distribution of fraud vs. non-fraud cases.

In **CA2**, we focus on additional EDA tasks that are essential for model creation:
- Imbalanced Data: Check how imbalanced is the dataset

- Feature Importance: Identifying which features have the most impact on predicting fraud.

- Outlier Detection: Detecting and handling outliers that could skew model performance.

In [19]:
## 
merged_profile = merged_df.profile_report(title='Merged Data Profiling Report', explorative=True)
merged_profile.to_file('merged_data_profiling_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

---

## **4. Model Workflow**

---

Phase 1: Baseline Experiments
Preprocessing: Basic encoding (e.g., one-hot encoding) and scaling.

Models: Logistic Regression, Random Forest, XGBoost.

Goal: Establish baseline performance metrics.

Phase 2: Advanced Preprocessing
Introduce techniques like SMOTE, RUS, ROS for handling imbalanced data.

Experiment with different encoding strategies (e.g., target encoding, frequency encoding).

Goal: Identify which preprocessing steps improve performance.

Phase 3: Feature Engineering and Selection
Create new features (e.g., time-based features, aggregated customer behavior).

Apply feature selection techniques (e.g., PCA, feature importance from tree-based models).

Goal: Optimize the feature set for the best-performing models.

Phase 4: Hyperparameter Tuning
Use Grid Search or Bayesian Optimization to fine-tune hyperparameters for the best-performing models.

Goal: Maximize model performance.

Phase 5: Final Pipeline
Combine the best preprocessing steps, feature engineering techniques, and models into a single pipeline.

Deploy the pipeline in your GUI application.

---

Phase 1: We defined the key metrics we will be using to determine the we will be using